### Assignment 09 : Colin Kirby

In [1]:
# Imports.
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame & OpenGL.
def init_window(width=800, height=600):
    pygame.init()
    pygame.display.set_mode((width, height), pygame.OPENGL | pygame.DOUBLEBUF)
    pygame.display.set_caption("Assignment 09 : Colin Kirby")

    # Create and Set up OpenGL Context.
    ctx = moderngl.create_context()
    ctx.enable(moderngl.CULL_FACE)
    ctx.enable(moderngl.DEPTH_TEST)

    return ctx

In [3]:
# Initialize Window.
ctx = init_window()

In [4]:
# Load Vertex Data for Platform, Teapot, & Light Source.
vertex_data_platform, _ = getObjectData("cube.obj", normal=True, texture=True)
vertex_data_teapot, _ = getObjectData("teapot_with_texCoords.obj", normal=True, texture=True)
vertex_data_light, _ = getObjectData("20_icosahedron.obj", normal=True)

Normal exists
texture exists
Normal exists
texture exists
Normal computed.


In [5]:
# Global VAO & Program references.
vao_platform = None
vao_teapot = None
vao_light = None
vao_skybox = None
program_platform = None
program_teapot = None
program_light = None
program_skybox = None

In [6]:
# Texture Loading Func.
def load_texture(filename):
    texture_img = pygame.image.load(filename)
    texture_data = pygame.image.tobytes(texture_img, "RGB", True)
    texture = ctx.texture(texture_img.get_size(), 3, texture_data)
    texture.build_mipmaps()
    texture.repeat_x = True
    texture.repeat_y = True
    return texture

In [7]:
# Load textures for Grass (Floor) & Gold (Teapot).
texture_floor = load_texture("grass.jpg")
texture_gold = load_texture("gold.jpg")

In [8]:
# Load Cube Map Texture from Provided Imgs.
def load_cube_map_texture():
    faces = [
        'Footballfield/posx.jpg',  # Right
        'Footballfield/negx.jpg',  # Left
        'Footballfield/posy.jpg',  # Top
        'Footballfield/negy.jpg',  # Bottom
        'Footballfield/posz.jpg',  # Front
        'Footballfield/negz.jpg',  # Back
    ]
    images = []
    for filename in faces:
        texture_img = pygame.image.load(filename).convert()
        flip_x, flip_y = (False, True) if 'posy' not in filename and 'negy' not in filename else (True, True)
        texture_img = pygame.transform.flip(texture_img, flip_x, flip_y)
        texture_data = pygame.image.tobytes(texture_img, "RGB", True)
        images.append((texture_img.get_size(), texture_data))
    size = images[0][0]
    cube_texture = ctx.texture_cube(size, 3, None)
    for i, (size, data) in enumerate(images):
        cube_texture.write(i, data)
    cube_texture.build_mipmaps()
    return cube_texture

cube_map_texture = load_cube_map_texture()

In [9]:
# Upload Vertex Data for Platform.
def set_vert_data_platform():
    global vao_platform, program_platform

    # Convert Vertex Data to NP Array.
    vertex_data_platform_np = np.array(vertex_data_platform, dtype='f4')
    vbo_platform = ctx.buffer(vertex_data_platform_np.tobytes())

    # Compile Shader Program.
    program_platform = ctx.program(
        vertex_shader=""" 
        #version 330
        in vec3 in_position;
        in vec3 in_normal;
        in vec2 in_uv;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        uniform mat3 normalMatrix;

        out vec3 frag_normal;
        out vec2 frag_uv;

        void main() {
            frag_normal = normalize(normalMatrix * in_normal);
            frag_uv = in_uv;
            gl_Position = proj * view * model * vec4(in_position, 1.0);
        }
        """,
        fragment_shader="""
        #version 330
        in vec3 frag_normal;
        in vec2 frag_uv;

        uniform sampler2D texture_diffuse;

        out vec4 frag_color;

        void main() {
            vec2 tiled_uv = frag_uv * 10.0; // Tiling factor
            vec3 tex_color = texture(texture_diffuse, tiled_uv).rgb;
            vec3 norm = normalize(frag_normal);

            vec3 up_color = vec3(0.8, 0.8, 0.8);   // Sky color
            vec3 down_color = vec3(0.2, 0.2, 0.2); // Ground color
            float f = (norm.y + 1.0) * 0.5;        // Map from [-1,1] to [0,1]
            vec3 hemi_light = mix(down_color, up_color, f);
            vec3 color = tex_color * hemi_light;

            frag_color = vec4(color, 1.0);
        }
        """
    )

    # Create the VAO for the Platform.
    vao_platform = ctx.vertex_array(
        program_platform,
        [
            (vbo_platform, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')
        ]
    )

In [10]:
# Upload Vertex Data for Teapot.
def set_vert_data_teapot():
    global vao_teapot, program_teapot, min_teapot_y

    # Convert Teapot Vertex Data to NP Array.
    vertex_data_teapot_np = np.array(vertex_data_teapot, dtype='f4')
    vbo_teapot = ctx.buffer(vertex_data_teapot_np.tobytes())

    # Compute min_teapot_y.
    vertex_count = len(vertex_data_teapot) // 8
    positions = vertex_data_teapot_np.reshape(vertex_count, 8)[:, :3]
    min_teapot_y = positions[:, 1].min()

    # Compile Shader Program for Environment Mapping.
    program_teapot = ctx.program(
        vertex_shader="""
        #version 330
        in vec3 in_position;
        in vec3 in_normal;
        in vec2 in_uv;

        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        uniform mat3 normalMatrix;

        out vec3 frag_pos;
        out vec3 frag_normal;
        out vec2 frag_uv;

        void main() {
            frag_pos = vec3(model * vec4(in_position, 1.0));
            frag_normal = normalize(normalMatrix * in_normal);
            frag_uv = in_uv;
            gl_Position = proj * view * vec4(frag_pos, 1.0);
        }
        """,
        fragment_shader="""
        #version 330
        in vec3 frag_pos;
        in vec3 frag_normal;
        in vec2 frag_uv;

        uniform vec3 eye_pos;
        uniform samplerCube cubemap;
        uniform sampler2D texture_diffuse;

        out vec4 frag_color;

        void main() {
            vec3 norm = normalize(frag_normal);
            vec3 V = normalize(eye_pos - frag_pos);
            vec3 R = reflect(-V, norm);

            vec3 env_color = texture(cubemap, R).rgb;
            vec3 base_color = texture(texture_diffuse, frag_uv).rgb;

            // Enhanced gold tint
            vec3 gold_tint = vec3(1.5, 1.0, 0.0);
            base_color = clamp(base_color * gold_tint, 0.0, 1.0);

            // Specular highlight calculation
            vec3 light_dir = normalize(vec3(0.0, 1.0, 0.0)); // Light from above
            vec3 half_vector = normalize(V + light_dir);
            float spec = pow(max(dot(norm, half_vector), 0.0), 64.0);
            vec3 specular = vec3(1.0) * spec;

            // Combine base color, environment reflection, and specular highlight
            vec3 final_color = mix(base_color, env_color, 0.7) + specular * 0.5;

            frag_color = vec4(final_color, 1.0);
        }
        """
    )

    # Create VAO for Teapot.
    vao_teapot = ctx.vertex_array(
        program_teapot,
        [
            (vbo_teapot, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')
        ]
    )

In [11]:
# Upload Vertex Data for Skybox.
def set_vert_data_skybox():
    global vao_skybox, program_skybox

    # Skybox Cube Vertices.
    skybox_vertices = np.array([
        # Positions for Skybox Cube.        
        -1.0,  1.0, -1.0,
        -1.0, -1.0, -1.0,
         1.0, -1.0, -1.0,
         1.0, -1.0, -1.0,
         1.0,  1.0, -1.0,
        -1.0,  1.0, -1.0,

        -1.0, -1.0,  1.0,
        -1.0, -1.0, -1.0,
        -1.0,  1.0, -1.0,
        -1.0,  1.0, -1.0,
        -1.0,  1.0,  1.0,
        -1.0, -1.0,  1.0,

         1.0, -1.0, -1.0,
         1.0, -1.0,  1.0,
         1.0,  1.0,  1.0,
         1.0,  1.0,  1.0,
         1.0,  1.0, -1.0,
         1.0, -1.0, -1.0,

        -1.0, -1.0,  1.0,
        -1.0,  1.0,  1.0,
         1.0,  1.0,  1.0,
         1.0,  1.0,  1.0,
         1.0, -1.0,  1.0,
        -1.0, -1.0,  1.0,

        -1.0,  1.0, -1.0,
         1.0,  1.0, -1.0,
         1.0,  1.0,  1.0,
         1.0,  1.0,  1.0,
        -1.0,  1.0,  1.0,
        -1.0,  1.0, -1.0,

        -1.0, -1.0, -1.0,
        -1.0, -1.0,  1.0,
         1.0, -1.0, -1.0,
         1.0, -1.0, -1.0,
        -1.0, -1.0,  1.0,
         1.0, -1.0,  1.0
    ], dtype='f4')

    vbo_skybox = ctx.buffer(skybox_vertices.tobytes())

    program_skybox = ctx.program(
        vertex_shader="""
        #version 330
        in vec3 in_position;

        uniform mat4 view;
        uniform mat4 proj;

        out vec3 frag_texcoord;

        void main() {
            frag_texcoord = in_position;
            vec4 pos = proj * view * vec4(in_position, 1.0);
            gl_Position = pos.xyww; // Set w component to 1.0
        }
        """,
        fragment_shader="""
        #version 330
        in vec3 frag_texcoord;
        out vec4 frag_color;

        uniform samplerCube cubemap;

        void main() {
            frag_color = texture(cubemap, frag_texcoord);
        }
        """
    )

    vao_skybox = ctx.vertex_array(
        program_skybox,
        [
            (vbo_skybox, '3f', 'in_position')
        ]
    )

In [12]:
# Initialize Vertex Data.
set_vert_data_platform()
set_vert_data_teapot()
set_vert_data_skybox()

In [13]:
# Projection Matrix.
fov = 45.0
width, height = 800, 600
aspect_ratio = width / height  
near_plane = 1.0  
far_plane = 100.0 
proj_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)

In [14]:
# Rendering Loop.
running = True
render_skybox = True

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_s:
                render_skybox = not render_skybox

    # Update Camera Position.
    time_in_seconds = pygame.time.get_ticks() / 1000.0
    angle = time_in_seconds * 0.5
    radius = 20.0
    eye_position = glm.vec3(
        radius * glm.sin(angle),
        2.0,  # Camera height
        radius * glm.cos(angle)
    )

    view_direction = glm.normalize(glm.vec3(0.0, 2.0, 10.0))
    target_position = eye_position + view_direction

    view_matrix = glm.lookAt(
        eye_position,
        glm.vec3(0.0, 0.5, 0.0),
        glm.vec3(0.0, 1.0, 0.0)
    )

    # Clear screen.
    ctx.clear(0.1, 0.1, 0.1, depth=1.0)

    # Render Skybox If Toggled.
    if render_skybox:
        ctx.disable(moderngl.DEPTH_TEST)
        view_matrix_skybox = glm.mat4(glm.mat3(view_matrix))
        program_skybox['view'].write(np.array(view_matrix_skybox, dtype='f4').T.tobytes())
        program_skybox['proj'].write(np.array(proj_matrix, dtype='f4').T.tobytes())
        cube_map_texture.use(location=0)
        program_skybox['cubemap'].value = 0
        vao_skybox.render(moderngl.TRIANGLES)
        ctx.enable(moderngl.DEPTH_TEST)

    # Render Platform.
    if vao_platform is not None and program_platform is not None:
        texture_floor.use(location=0)
        program_platform['texture_diffuse'].value = 0
        # Adjust platform position to sit on the XZ plane
        model_matrix_platform = glm.translate(glm.mat4(1.0), glm.vec3(0.0, -0.025, 0.0)) * glm.scale(glm.mat4(1.0), glm.vec3(100.0, 0.05, 100.0))
        normal_matrix_platform = glm.transpose(glm.inverse(glm.mat3(model_matrix_platform)))
        program_platform['model'].write(np.array(model_matrix_platform, dtype='f4').T.tobytes())
        program_platform['normalMatrix'].write(np.array(normal_matrix_platform, dtype='f4').T.tobytes())
        program_platform['view'].write(np.array(view_matrix, dtype='f4').T.tobytes())
        program_platform['proj'].write(np.array(proj_matrix, dtype='f4').T.tobytes())
        vao_platform.render(moderngl.TRIANGLES)

    # Render Teapot.
    if vao_teapot is not None and program_teapot is not None:
        program_teapot['eye_pos'].write(np.array(eye_position, dtype='f4').tobytes())
        cube_map_texture.use(location=0)
        texture_gold.use(location=1)
        program_teapot['cubemap'].value = 0
        program_teapot['texture_diffuse'].value = 1

        # Compute Teapot Position.
        scale_factor = 0.3
        y_pos = - (min_teapot_y * scale_factor)
        model_matrix_teapot = glm.translate(glm.mat4(1.0), glm.vec3(0.0, y_pos, 0.0)) * glm.scale(glm.mat4(1.0), glm.vec3(scale_factor))
        normal_matrix_teapot = glm.transpose(glm.inverse(glm.mat3(model_matrix_teapot)))

        program_teapot['model'].write(np.array(model_matrix_teapot, dtype='f4').T.tobytes())
        program_teapot['normalMatrix'].write(np.array(normal_matrix_teapot, dtype='f4').T.tobytes())
        program_teapot['view'].write(np.array(view_matrix, dtype='f4').T.tobytes())
        program_teapot['proj'].write(np.array(proj_matrix, dtype='f4').T.tobytes())
        vao_teapot.render(moderngl.TRIANGLES)

    # Swap Display Buffers.
    pygame.display.flip()

# Clean Up.
pygame.quit()
